In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from downloader import download_all
from preprocess import preprocess_files
from process_headers import process_headers, value_fixes
import pickle

import dotenv
dotenv.load_dotenv('.env', override=True)

True

In [3]:
filenames = download_all()
filenames

{1999: 'downloads/lamas-muni-1999.xls',
 2000: 'downloads/lamas-muni-2000.xls',
 2001: 'downloads/lamas-muni-2001.xls',
 2002: 'downloads/lamas-muni-2002.xls',
 2003: 'downloads/lamas-muni-2003.xls',
 2004: 'downloads/lamas-muni-2004.xls',
 2005: 'downloads/lamas-muni-2005.xls',
 2006: 'downloads/lamas-muni-2006.xls',
 2007: 'downloads/lamas-muni-2007.xls',
 2008: 'downloads/lamas-muni-2008.xls',
 2009: 'downloads/lamas-muni-2009.xls',
 2010: 'downloads/lamas-muni-2010.xls',
 2011: 'downloads/lamas-muni-2011.xls',
 2012: 'downloads/lamas-muni-2012.xls',
 2013: 'downloads/lamas-muni-2013.xls',
 2014: 'downloads/lamas-muni-2014.xls',
 2015: 'downloads/lamas-muni-2015.xls',
 2016: 'downloads/lamas-muni-2016.xlsx',
 2017: 'downloads/lamas-muni-2017.xlsx',
 2018: 'downloads/lamas-muni-2018.xlsx',
 2019: 'downloads/lamas-muni-2019.xlsx',
 2020: 'downloads/lamas-muni-2020.xlsx',
 2021: 'downloads/lamas-muni-2021.xlsx',
 2022: 'downloads/lamas-muni-2022.xlsx'}

In [4]:
data = process_headers(preprocess_files(filenames)).results()[0][0]
# data[:200]
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

1999 downloads/lamas-muni-1999.xls
PROCESSING 1999 'עיריות' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
PROCESSING 1999 'מועצות מקומיות' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
PROCESSING 1999 'מועצות אזוריות' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
2000 downloads/lamas-muni-2000.xls
PROCESSING 2000 'נתונים פיזיים - עיריות ומ.מקומי' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
PROCESSING 2000 'נתונים כספיים - עיריות ומ.מקומי' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
PROCESSING 2000 'מועצות אזוריות' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
2001 downloads/lamas-muni-2001.xls
PROCESSING 2001 'נתונים פיזיים - עיריות ומ.מקומי' Config(header_rows=1, extend_headers_top=0, extend_headers_bottom=0, skip=False)
PROCESSING 2001 'נתונים כספיים - עיריות ומ.מקומי' Conf

/Users/adam/.pyenv/versions/3.12.1/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


PROCESSING 2017 'נתונים פיזיים ונתוני אוכלוסייה' Config(header_rows=4, extend_headers_top=2, extend_headers_bottom=0, skip=False)
PROCESSING 2017 'נתוני תקציב' Config(header_rows=5, extend_headers_top=2, extend_headers_bottom=0, skip=False)
BAD FLOATS {'=Q94/R94100'}
2018 downloads/lamas-muni-2018.xlsx
PROCESSING 2018 'נתונים פיזיים ונתוני אוכלוסייה' Config(header_rows=4, extend_headers_top=2, extend_headers_bottom=0, skip=False)
PROCESSING 2018 'נתוני תקציב' Config(header_rows=5, extend_headers_top=0, extend_headers_bottom=0, skip=False)
2019 downloads/lamas-muni-2019.xlsx
PROCESSING 2019 'נתונים פיזיים ונתוני אוכלוסייה' Config(header_rows=4, extend_headers_top=2, extend_headers_bottom=0, skip=False)
PROCESSING 2019 'נתוני תקציב' Config(header_rows=5, extend_headers_top=1, extend_headers_bottom=0, skip=False)
2020 downloads/lamas-muni-2020.xlsx
PROCESSING 2020 'נתונים פיזיים ונתוני אוכלוסייה' Config(header_rows=4, extend_headers_top=2, extend_headers_bottom=0, skip=False)
PROCESSING 2

In [5]:
if False:
    import dataflows as DF
    from collections import Counter
    from dataflows_airtable import load_from_airtable, dump_to_airtable, AIRTABLE_ID_FIELD
    import pickle

    with open('data.pickle', 'rb') as f:
        data = pickle.load(f)

    print('GETTING MAPPING', len(data))
    mapping = DF.Flow(
        data,
        DF.select_fields(['header', 'orig_header']),
        DF.update_resource(-1, name='headers'),
        DF.join_with_self('headers', ['orig_header', 'header'], dict(header=None, orig_header=None)),
    ).results()[0][0]
    print('GOT MAPPING', len(mapping))

    DF.Flow(
        load_from_airtable('apptGe94qTaLjk5Fr', 'Header Mapping', 'Grid view', 'env://AIRTABLE_API_KEY'),
        DF.select_fields([AIRTABLE_ID_FIELD, 'header', 'orig_header']),
        DF.update_resource(-1, name='current'),
        mapping,
        DF.update_resource(-1, name='headers', path='data/headers.csv'),
        DF.add_field('count', 'integer', 1),
        DF.join('current', ['orig_header'], 'headers', ['orig_header'], {AIRTABLE_ID_FIELD:None}, mode='full-outer'),
        DF.set_type('count', transform=lambda v: 1 if v else 0),
        dump_to_airtable({
            ('apptGe94qTaLjk5Fr', 'Header Mapping'): {
                'resource-name': 'headers',
            }
        }, 'env://AIRTABLE_API_KEY'),
    ).process()


In [6]:
if True:
    import dataflows as DF
    from collections import Counter
    from dataflows_airtable import load_from_airtable, dump_to_airtable, AIRTABLE_ID_FIELD

    with open('data.pickle', 'rb') as f:
        data = pickle.load(f)

    mapping = DF.Flow(
        data,
        DF.select_fields(['header', 'year']),
        DF.update_resource(-1, name='headers'),
        DF.join_with_self('headers', ['header', 'year'], dict(
            header=None,
            year=None,
        )),
        DF.sort_rows('{year}'),
        DF.join_with_self('headers', ['header'], dict(
            header=None,
            years=dict(name='year', aggregate='array'),
            count=dict(aggregate='count'),
            max_year=dict(name='year', aggregate='max'),
            min_year=dict(name='year', aggregate='min')
        )),
    ).results()[0][0]
    print(mapping[:10])

    DF.Flow(
        load_from_airtable('apptGe94qTaLjk5Fr', 'Stats', 'Grid view', 'env://AIRTABLE_API_KEY'),
        DF.select_fields([AIRTABLE_ID_FIELD, 'header']),
        DF.update_resource(-1, name='current'),
        mapping,
        DF.update_resource(-1, name='headers', path='data/headers.csv'),
        DF.join('current', ['header'], 'headers', ['header'], {AIRTABLE_ID_FIELD:None}, mode='full-outer'),
        DF.set_type('count', transform=lambda v: v or 0),
        DF.set_type('years', type='string', transform=lambda v: ','.join(sorted(map(str,v))) if v else ''),
        dump_to_airtable({
            ('apptGe94qTaLjk5Fr', 'Stats'): {
                'resource-name': 'headers',
            }
        }, 'env://AIRTABLE_API_KEY'),
    ).process()


/Users/adam/Code/dataflows/dataflows/processors/join.py:189: UserWarning: For the `join` processor the `full=True` flag is deprecated. Please use the "mode" parameter instead.
  warnings.warn(


[{'header': 'אפיון טופוגרפי - גובה מעל פני הים, במטרים (טווח הגבהים)', 'count': 5, 'max_year': 2017, 'min_year': 2004, 'years': [2004, 2005, 2006, 2016, 2017]}, {'header': 'אפיון טופוגרפי - גובה מעל פני הים, במטרים (מינימום)', 'count': 3, 'max_year': 2006, 'min_year': 2004, 'years': [2004, 2005, 2006]}, {'header': 'אפיון טופוגרפי - גובה מעל פני הים, במטרים (ממוצע הגבהים)', 'count': 3, 'max_year': 2017, 'min_year': 2006, 'years': [2006, 2016, 2017]}, {'header': 'אפיון טופוגרפי - גובה מעל פני הים, במטרים (מקסימום)', 'count': 5, 'max_year': 2017, 'min_year': 2004, 'years': [2004, 2005, 2006, 2016, 2017]}, {'header': 'אפיון טופוגרפי - ממוצע ציוני תקן של סטיית התקן של הגבהים, ושל שיפוע הממוצע (אשכול)', 'count': 3, 'max_year': 2017, 'min_year': 2006, 'years': [2006, 2016, 2017]}, {'header': 'אפיון טופוגרפי - ממוצע ציוני תקן של סטיית התקן של הגבהים, ושל שיפוע הממוצע (דירוג)', 'count': 3, 'max_year': 2017, 'min_year': 2006, 'years': [2006, 2016, 2017]}, {'header': 'אפיון טופוגרפי - ממוצע ציוני

In [3]:
### WRITE TO DB

import dataflows as DF

with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

def count(rows):
    for i, row in enumerate(rows):
        if row['name'] == 'רעננה' and 'עלות עבודה' in row['orig_header']:
            print(row)
        yield row
        if i in (1, 10, 100, 1000, 2000, 5000) or i % 100000 == 0:
            print(i)


dp, _ = DF.Flow(
    data,
    value_fixes(),
    # DF.set_type('orig_header', type='string', transform=lambda v: '/'.join(v)),
    DF.update_resource(-1, name='lamas'),
    count,
    # DF.dump_to_sql(dict(
    #     lamas_muni={
    #         'resource-name': 'lamas',
    #         # 'mode': 'rewrite',
    #         # 'indexes_fields': [
    #         #     ['name', 'year', 'header'],
    #         # ],
    #     }
    # ), batch_size=1000),
    DF.dump_to_path('db_bkp')
).process()
dp.descriptor


0
1
10
100
1000
2000
5000
100000
200000
300000
{'year': 2004, 'name': 'רעננה', 'header': 'הוצאות בתקציב הרגיל - סה"כ הוצאות לעלות עבודה (אלפי ש"ח)', 'value': '184137', 'filename': 'downloads/lamas-muni-2004.xls', 'min_year': None, 'orig_header': 'סה"כ הוצאות של הרשות לעלות עבודה (אלפי ש"ח)'}
{'year': 2004, 'name': 'רעננה', 'header': 'הוצאות בתקציב הרגיל - סה"כ הוצאות לעלות עבודה (אחוז שינוי ריאלי)', 'value': '1.314963542855474', 'filename': 'downloads/lamas-muni-2004.xls', 'min_year': None, 'orig_header': 'סה"כ הוצאות של הרשות לעלות עבודה - אחוז שינוי ראלי לעומת'}
{'year': 2004, 'name': 'רעננה', 'header': 'הוצאות בתקציב הרגיל - עלות עבודה לחינוך (אלפי ש"ח)', 'value': '62682', 'filename': 'downloads/lamas-muni-2004.xls', 'min_year': None, 'orig_header': 'הוצאות של הרשות לעלות עבודה לחינוך (אלפי ש"ח)'}
{'year': 2004, 'name': 'רעננה', 'header': 'הוצאות בתקציב הרגיל - עלות עבודה לחינוך (אחוז שינוי ריאלי)', 'value': '-4.470719136964685', 'filename': 'downloads/lamas-muni-2004.xls', 'min_yea

{'profile': 'data-package',
 'resources': [{'name': 'lamas',
   'path': 'res_1.csv',
   'profile': 'tabular-data-resource',
   'schema': {'fields': [{'name': 'year',
      'type': 'integer',
      'format': 'default'},
     {'name': 'name', 'type': 'string', 'format': 'default'},
     {'name': 'header', 'type': 'string', 'format': 'default'},
     {'name': 'value', 'type': 'string', 'format': 'default'},
     {'name': 'filename', 'type': 'string', 'format': 'default'},
     {'name': 'min_year', 'type': 'integer', 'format': 'default'},
     {'name': 'orig_header', 'type': 'string', 'format': 'default'}],
    'missingValues': ['']},
   'encoding': 'utf-8',
   'format': 'csv',
   'dialect': {'lineTerminator': '\r\n',
    'delimiter': ',',
    'doubleQuote': True,
    'quoteChar': '"',
    'skipInitialSpace': False,
    'header': True,
    'caseSensitiveHeader': False},
   'bytes': 471582565,
   'hash': '4aa9f08bb419d9097c0c6cd5d24c7004'}],
 'bytes': 471584115,
 'count_of_rows': 2027002,
 